In [1]:
import random
import pandas as pd
from tqdm import tqdm
from collections import Counter

import numpy as np
from scipy import stats
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)
plotly.tools.set_credentials_file(username='andreffs18', api_key='')

import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()


from optimization import (get_flights_schedule, print_schedule, schedule_cost, 
                          random_optimize, 
                          hillclimb_optimize, 
                          annealing_optimize, 
                          genetic_optimize)

In [2]:
people = [('Seymour', 'BOS'),
          ('Franny', 'DAL'),
          ('Zooey', 'CAK'),
          ('Walt', 'MIA'),
          ('Buddy', 'ORD'),
          ('Les', 'OMA')]
# Laguardia
destination = 'LGA'

flights = get_flights_schedule()

In [3]:
# Get highest and lowest pairs for Seymour
origin = dict(people).get('Seymour')
highest_cost = 0
highest_pair = None
lowest_cost = 99999
lowest_pair = None
for d in range(len(flights[(origin, destination)])):
    for a in range(len(flights[(destination, origin)])):
        cost = schedule_cost([d, a], people, flights, destination)
        if cost > highest_cost:
            highest_pair = (d, a)
            highest_cost = cost
        if cost < lowest_cost:
            lowest_pair = (d, a)
            lowest_cost = cost
print_schedule((highest_pair), people, flights, destination)
print("Cost Function value: {}\nHighest Pair: {}".format(highest_cost, highest_pair))
print_schedule((lowest_pair), people, flights, destination)
print("Cost Function value: {}\nLowest Pair: {}".format(lowest_cost, lowest_pair))

   Seymour       BOS  9:45-11:50 $172  8:23-10:28 $149 	(Total: $321)
Cost Function value: 371
Highest Pair: (2, 1)
   Seymour       BOS 11:16-13:29 $ 83 15:25-16:58 $ 62 	(Total: $145)
Cost Function value: 145
Lowest Pair: (3, 6)


In [4]:
# Compare agains random search
# domain = [(0, 8)] * (len(people) * 2) 
domain = [(0, 8), (0, 8)]
random_optimize(domain, people, flights, guesses=9999)

100%|██████████| 9999/9999 [00:01<00:00, 6220.69it/s]


[3.0, 6.0]

In [5]:
print_schedule((3.0, 6.0), people, flights, destination)

   Seymour       BOS 11:16-13:29 $ 83 15:25-16:58 $ 62 	(Total: $145)


In [6]:
domain = [(0, 8)] * (len(people) * 2)
hillclimb_optimize(domain, people, flights)

[8, 0, 5, 0, 8, 5, 8, 0, -2, 0, 8, 6]

In [7]:
domain = [(0, 8)] * (len(people) * 2)
annealing_optimize(domain, people, flights)

[7.0, 3.0, 5.0, 3.0, 7.0, 5.0, 3.0, 6.0, 6.0, 3.0, 7.0, 6.0]

In [8]:
print_schedule((7.0, 3.0), people, flights, destination)

   Seymour       BOS 17:11-18:30 $108 10:33-12:03 $ 74 	(Total: $182)


In [6]:
domain = [(0, 8)] * (len(people) * 2)
genetic_optimize(domain, people, flights, popsize=100, step=2, mutprob=0.25, elite=0.15, maxiter=1000, repeat_min_score=50)

  6%|▌         | 58/1000 [00:03<00:55, 16.92it/s]


[1, 6, 0, 5, 2, 6, 2, 6, 1, 5, 2, 7]

In [7]:
print_schedule((1.0, 6.0), people, flights, destination)

   Seymour       BOS  8:04-10:11 $ 95 15:25-16:58 $ 62 	(Total: $157)
